- incorporate validation method
- vaildation seed change
- modify mistake

In [1]:
import sys
!cp ../input/rapids/rapids.0.15.0 /opt/conda/envs/rapids.tar.gz
!cd /opt/conda/envs/ && tar -xzvf rapids.tar.gz > /dev/null
sys.path = ["/opt/conda/envs/rapids/lib/python3.7/site-packages"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib/python3.7"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib"] + sys.path
!cp /opt/conda/envs/rapids/lib/libxgboost.so /opt/conda/lib/

In [2]:
import os
import warnings
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from tqdm import tqdm_notebook as tqdm
from category_encoders import CountEncoder
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import KFold, StratifiedKFold
sys.path.append('../input/multilabelstraifier/')
from ml_stratifiers import MultilabelStratifiedKFold
warnings.filterwarnings('ignore')

from sklearn.kernel_approximation import Nystroem
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LogisticRegression
from sklearn.kernel_ridge import KernelRidge
from cuml.svm import SVC, SVR

# preprocess

In [3]:
DATA_DIR = '/kaggle/input/lish-moa/'
train = pd.read_csv(DATA_DIR + 'train_features.csv')
targets = pd.read_csv(DATA_DIR + 'train_targets_scored.csv')
test = pd.read_csv(DATA_DIR + 'test_features.csv')
drug = pd.read_csv(DATA_DIR + 'train_drug.csv')

In [4]:
target_feats = [ i for i in targets.columns if i != "sig_id"]
g_feats = [i for i in train.columns if "g-" in i]
c_feats = [i for i in train.columns if "c-" in i]

In [5]:
noncons_train_index = train[train.cp_type=="ctl_vehicle"].index
cons_train_index = train[train.cp_type!="ctl_vehicle"].index
noncons_test_index = test[test.cp_type=="ctl_vehicle"].index
cons_test_index = test[test.cp_type!="ctl_vehicle"].index

y = targets.drop("sig_id", axis=1).copy()
test = test[test.index.isin(cons_test_index)].reset_index(drop=True)
train = train[train.index.isin(cons_train_index)].reset_index(drop=True)
targets = targets[targets.index.isin(cons_train_index)].reset_index(drop=True)
fn_targets = targets.drop("sig_id", axis=1).copy().to_numpy()

In [6]:
# https://www.kaggle.com/c/lish-moa/discussion/195195
N_SPLITS = 5
seed = 14

folds = []
    
# LOAD FILES
train_score = targets.merge(drug, on='sig_id', how='left') 

# LOCATE DRUGS
vc = train_score.drug_id.value_counts()
vc1 = vc.loc[vc <= 18].index.sort_values()
vc2 = vc.loc[vc > 18].index.sort_values()
    
# STRATIFY DRUGS 18X OR LESS
dct1 = {}; dct2 = {}
skf = MultilabelStratifiedKFold(n_splits = N_SPLITS, shuffle = True, random_state = seed)
tmp = train_score.groupby('drug_id')[target_feats].mean().loc[vc1]
for fold,(idxT,idxV) in enumerate(skf.split(tmp,tmp[target_feats])):
    dd = {k:fold for k in tmp.index[idxV].values}
    dct1.update(dd)

# STRATIFY DRUGS MORE THAN 18X
skf = MultilabelStratifiedKFold(n_splits = N_SPLITS, shuffle = True, random_state = seed)
tmp = train_score.loc[train_score.drug_id.isin(vc2)].reset_index(drop = True)
for fold,(idxT,idxV) in enumerate(skf.split(tmp,tmp[target_feats])):
    dd = {k:fold for k in tmp.sig_id[idxV].values}
    dct2.update(dd)
    
# ASSIGN FOLDS
train_score['fold'] = train_score.drug_id.map(dct1)
train_score.loc[train_score.fold.isna(),'fold'] = train_score.loc[train_score.fold.isna(),'sig_id'].map(dct2)
train_score.fold = train_score.fold.astype('int8')
folds.append(train_score.fold.values)
    
np.array(folds)

array([[3, 0, 2, ..., 4, 1, 0]], dtype=int8)

In [7]:
target_feats = [i for i in targets.columns if i != "sig_id"]

check_dist = targets.copy()
check_dist["fold"] = np.array(folds).reshape(-1,1)

noncons_feats = []
for ind, i in enumerate(target_feats):
    if np.max(check_dist.loc[:,[i]+["fold"]].groupby("fold").sum()).values[0] == check_dist[i].sum() and check_dist[i].sum()>=5:
        noncons_feats.append(ind)

# Feature engineering

In [8]:
for i in c_feats + g_feats:
    ss = preprocessing.StandardScaler()
    ss.fit(train[i].values.reshape(-1,1))
    train[i] = ss.transform(train[i].values.reshape(-1,1))
    test[i] = ss.transform(test[i].values.reshape(-1,1))

In [9]:
def fe_simple(df, remove_features):
    tmp = df.copy()
    tmp.drop(remove_features, axis=1, inplace=True)
    tmp.loc[:, 'cp_dose'] = tmp.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})
    return tmp

remove_features = ["cp_type" , "sig_id"]
train_fn = fe_simple(train, remove_features)
test_fn = fe_simple(test, remove_features)
train_fn["fold"] = np.array(folds).reshape(-1,1)

print(train_fn.shape, test_fn.shape)

(21948, 875) (3624, 874)


In [10]:
fn_train = train_fn.copy().to_numpy()
fn_test = test_fn.copy().to_numpy()

# modelling

In [11]:
N_STARTS = 1

def modelling_svm(tr, ta, te):
    svm0_oof = np.zeros([len(tr), ta.shape[1]])
    svm0_test = np.zeros([len(te), ta.shape[1]])

    svm1_test = np.zeros([len(te),ta.shape[1]])
    svm1_oof = np.zeros([ta.shape[0],ta.shape[1]]) 

    for ind in tqdm(range(ta.shape[1])):
        ind_target_sum = ta[:, ind].sum()
        if ind_target_sum >= N_SPLITS and ind not in noncons_feats:
            for seed in range(N_STARTS):
                for n in range(N_SPLITS):
                    val_index = tr[:,-1] == n
                    train_index = tr[:,-1] != n
                    x_tr, x_val = tr[train_index], tr[val_index]
                    y_tr, y_val = ta[train_index,ind], ta[val_index,ind]
                    x_tr = np.delete(x_tr, -1, 1)
                    x_val = np.delete(x_val, -1, 1)
                    model = SVC(C = 40, cache_size = 2000)
                    model.fit(x_tr, y_tr)
                    svm0_test[:, ind] += model.decision_function(te) / (N_SPLITS * N_STARTS)
                    svm0_oof[val_index, ind] += model.decision_function(x_val) / N_STARTS
                    
            for seed in range(N_STARTS):
                for n in range(N_SPLITS):
                    val_index = tr[:,-1] == n
                    train_index = tr[:,-1] != n
                    x_tr, x_val = svm0_oof[train_index, ind].reshape(-1, 1), svm0_oof[val_index, ind].reshape(-1, 1)
                    y_tr, y_val = ta[train_index,ind], ta[val_index,ind]

                    model = LogisticRegression(C = 35, max_iter = 1000)
                    model.fit(x_tr, y_tr)
                    svm1_test[:, ind] += model.predict_proba(svm0_test[:, ind].reshape(-1, 1))[:, 1] / (N_SPLITS * N_STARTS)
                    svm1_oof[val_index, ind] += model.predict_proba(x_val)[:, 1] / N_STARTS

        score1 = log_loss(ta[:, ind], svm0_oof[:, ind])
        score2 = log_loss(ta[:, ind], svm1_oof[:, ind])
        print('SVM Target ind {} score {}: {}'.format(ind, score1, score2))

    return svm1_test, svm1_oof

In [12]:
svm1_test, svm1_oof = modelling_svm(fn_train, fn_targets, fn_test)

SVM Target ind 0 score 0.0267522871657328: 0.004784850191536545
SVM Target ind 1 score 0.028325951116658196: 0.006774866422310902
SVM Target ind 2 score 0.0377679348222106: 0.008627308176932658
SVM Target ind 3 score 0.2990400893375156: 0.048465174073207294
SVM Target ind 4 score 0.4736841911390064: 0.0714749840350949
SVM Target ind 5 score 0.11487887210902842: 0.02234108130431479
SVM Target ind 6 score 0.0849778533499726: 0.017247776995837138
SVM Target ind 7 score 0.15112004990447125: 0.027869454031235186
SVM Target ind 8 score 0.018893542751029866: 0.004323261031128093
SVM Target ind 9 score 0.41344595459963224: 0.06089158561279115
SVM Target ind 10 score 0.56654328048957: 0.08236960589630517
SVM Target ind 11 score 0.0798682983062561: 0.017925761428782112
SVM Target ind 12 score 0.0110156476564788: 0.0110156476564788
SVM Target ind 13 score 0.06609921068559742: 0.013795053767412382
SVM Target ind 14 score 0.0188839674111058: 0.004892597137094785
SVM Target ind 15 score 0.0188839674

In [13]:
check_svm1 = np.zeros([y.shape[0], y.shape[1]])
check_svm1[cons_train_index,:] = svm1_oof
print('OOF log loss: ', log_loss(np.ravel(y), np.ravel(check_svm1)))

OOF log loss:  0.016892971920218523


In [14]:
pd.DataFrame(check_svm1).to_csv("svm_newval_oof.csv", index=False)

# submission

In [15]:
sub = pd.read_csv(DATA_DIR + 'sample_submission.csv')

sub.loc[cons_test_index,target_feats] =  svm1_test 
sub.loc[noncons_test_index,target_feats] = 0
sub.to_csv('submission.csv', index=False)